In [1]:
import pandas as pd

In [2]:
english_definitions = pd.read_parquet("data/english_definitions.parquet")

In [3]:
english_definitions.head()

,index,characters,pinyin,definition
0,2,八,bā,eight; 8
1,3,爸爸,bàba,"(informal) father; CL:個|个[gè],位[wèi]"
2,4,吧,ba,(modal particle indicating polite suggestion);...
3,5,白,bái,white; snowy; pure; bright; empty; blank; plai...
4,6,百,bǎi,hundred; numerous; all kinds of; surname Bai


In [4]:
from gemini import translate_definitions_spanish

# send batches to gemini
n = 20  
batchs_ready = []
number_batch = 1

for start in range(0, len(english_definitions), n):
    batch = english_definitions.iloc[start : start + n]
    fixed_batched = translate_definitions_spanish(batch[:20].to_dict('records'))

    batchs_ready.append(fixed_batched)

    print("Finished batch: ", number_batch)
    number_batch+=1

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2\nPlease retry in 54.400991795s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-pro'}, 'quotaValue': 

In [ ]:
spanish_definitions = pd.DataFrame([item for list_item in batchs_ready for item in list_item ])

chinese_spanish_vocabulary  = english_definitions.drop(columns='definition', axis=1).merge(spanish_definitions, how="left", left_on="index" , right_on="character_ID").drop(columns="character_ID")
chinese_spanish_vocabulary = chinese_spanish_vocabulary.sort_values('index', ascending=True).reset_index(drop=True)

NameError: name 'pd' is not defined

# Save the data

In [ ]:
chinese_spanish_vocabulary.to_parquet('data/chinese_spanish_vocabulary.parquet')